In [2]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv

# Using STABLE ID key to find what versioned IDs these point to in different Ensembl releases

## results are of how many version IDs are different given same stable ID within the same assembly and same source (ENSEMBL)



## goal: Ensembl Biomart stable ID between 37 and 38 - are stable IDs enough? 
> Do peptide/rna/dna sequences differ when I pull versions + sequences?

## UKB HUMAN_9606_idmapping file has stable ID mapping to ensembl for 38, but is this enough for 37? Compare to the file I made from biomart


## from uniprot ID_mapping.dat file
- HUMAN_9606_idmapping_grepCCDS.txt
    - does CCDS mapping the same between CCDS source 37 and 38 mapping?
    
- HUMAN_9606_idmapping_grepEnsembl.txt
    - does stable ID mapping between 37 and 38 have consequences?
    
- HUMAN_9606_idmapping_grepRefSeq.txt
    - does this match the CCDS source mapping for refseq?


## from biomart (19_09_04 ipynb created files)
1. (version and stable ID download) biomartGRCh37p.13_v97_filterCCDSIDonly.txt
2. (version and stable ID download) biomartGRCh38p.12_v97_filterCCDSIDonly.txt
- checkMatch = pd.read_csv("StableIDs_allMatched_btw_GRCh37_38_ensemblv97_22202.csv")
    - CONFIRMED TO MATCH between 37 and 38 on ENST, ENSG, ENSP, UKB ID, CCDS
- (downloaded) biomart_CCDSIDonly_GRCh37_v97_peptideFasta.txt also confirmed this matches v85 peptides [v85 == v97 peptide fasta using diff in bash]
- (downloaded) v92, v94, v96, v97 peptide fastas 


**COMPARE THE 22,202 STABLE IDS WITH VERSION FROM ENSEMBL LINKED TO STABLE UKB WITH THE UKB ISOFORM LINKED TO STABLE ENSEMBL IDS IN THE  HUMAN_9606_idmapping_grepEnsembl.txt **


---
---
---
---


In [1]:
# first attempt all code and notes in cell below: 


-------------------------------------------
# [PREVIOUS KEY ID ONLY ENSG_ENST_ENSP] 

## FROM 8.18.19: total 22,193 unique stable_id overlapping 37 and 38 

### from GRCh37 v97 file with 23,334 rows
- 22,205 match stableID overlap key
- 1,129 did not match 
- adds up to total = 23,334 QC
- 12 rows more that the length of unique overlap stable_ID list

### from GRCh38 v97 file with 38,719 rows
- 22,203 match stableID overlap key
- 16,516 did not match
- 10 rows more than length of unique overlapping stable IDs

## PROBLEM CAME from only using stable KEY with ENSG_ENST_ENSP:  the stable ID # 22,193 is less than the stable IDs matched in the versioned v97 GRCh37 and 38 file- 

- TF37: 
     - 6 unique stable IDs point to 18 unique UKB IDs
     
**redundancy is in pointing to UKB**

- TF38: 
    - 4 unique stable IDs point to 14 unique UKB IDs

**redundancy is in pointing to UKB**

- solution is to drop the none stable id or version id rows and then check how many unique stable IDs are in each file then do the version comparison 



## TODO-1: test differences in version of IDs that come from ENSG_ENST_ENSP matched stable IDs between the assemblies

- given 22,193 unique ENST_ENSG_ENSP IDs (combined rows), need to quantify the extent of version differences

```python

# packages
import os
import sys
import numpy as np
import pandas as pd
import csv

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

sys.path.append('/Users/mariapalafox/Desktop/Toolbox')
from all_funx import *

os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/BIOMART_ENSEMBL/")
print(os.listdir())

# files downloaded from biomart, specifics of file 
ccds37 = "biomartGRCh37p.13_v97_filterCCDSIDonly.txt"
ccds38 = "biomartGRCh38p.12_v97_filterCCDSIDonly.txt"
version37 = pd.read_csv(ccds37,sep='\t')
version38 = pd.read_csv(ccds38,sep='\t')

# matching stable ID file between assemblies
stableMatch = "StableIDs_ENST_ENSG_ENSP_v97ensembl_GRCh37_matchGRCh38_22193_fromBiomartCCDSonlyfilter.csv"
stable = pd.read_csv(stableMatch)

# drop Nan values reset index
version37.dropna(inplace=True)
version38.dropna(inplace=True)

version37.reset_index(drop=True,inplace=True)
version38.reset_index(drop=True,inplace=True)

describeMe(version37)
describeMe(version38)
describeMe(stable)

describeMe(version37)
describeMe(version38)
describeMe(stable)

##### results of describeMe #####
(23334, 9)
Index(['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Protein stable ID',
       'Protein stable ID version', 'CCDS ID', 'UniProtKB/Swiss-Prot ID',
       'Transcript having exact match between ENSEMBL and HAVANA ID'],
      dtype='object')
Gene stable ID                                                 0
Gene stable ID version                                         0
Transcript stable ID                                           0
Transcript stable ID version                                   0
Protein stable ID                                              0
Protein stable ID version                                      0
CCDS ID                                                        0
UniProtKB/Swiss-Prot ID                                        0
Transcript having exact match between ENSEMBL and HAVANA ID    0
dtype: int64
(38719, 8)
Index(['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Protein stable ID',
       'Protein stable ID version', 'CCDS ID', 'UniProtKB/Swiss-Prot ID'],
      dtype='object')
Gene stable ID                  0
Gene stable ID version          0
Transcript stable ID            0
Transcript stable ID version    0
Protein stable ID               0
Protein stable ID version       0
CCDS ID                         0
UniProtKB/Swiss-Prot ID         0
dtype: int64
(22193, 3)
Index(['ENST', 'Gene stable ID_x', 'Protein stable ID_x'], dtype='object')
ENST                   0
Gene stable ID_x       0
Protein stable ID_x    0
dtype: int64
################

# create stable_id for mapping to stableMatch IDs to versioned rows in 37 and 38 file 
version37['stable_id'] = version37['Gene stable ID'].astype(str) + \
'_' + version37['Transcript stable ID'].astype(str) + \
'_' + version37['Protein stable ID'].astype(str)

version38['stable_id'] = version38['Gene stable ID'].astype(str) + \
'_' + version38['Transcript stable ID'].astype(str) + \
'_' + version38['Protein stable ID'].astype(str)


stable['stable_id'] = stable['Gene stable ID_x'].astype(str) + \
'_' + stable['ENST'].astype(str) + \
'_' + stable['Protein stable ID_x'].astype(str)

# turn unique stable id key into list for funx
stableList = list(stable.stable_id)

# label each 37 and 38 version file with True False 
# if stable_id is in the unique set of ID from my stable ID overlap between assemblies analysis
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    return df

TF37 = addcolumnconditional(stableList, version37, "stable_id", "37match38stable_id")
TF38 = addcolumnconditional(stableList, version38, "stable_id", "37match38stable_id")


checkColumnValues(TF37, "37match38stable_id")
checkColumnValues(TF38, "37match38stable_id")
##### results checkColumnValues #######
37match38stable_id  Count
0               True  22205
1              False   1129

37match38stable_id  Count
0               True  22203
1              False  16516
######################
```

## total 22,193 unique stable_id overlapping 37 and 38 

### from GRCh37 v97 file with 23,334 rows
- 22,205 match stableID overlap key
- 1,129 did not match 
- adds up to total = 23,334 QC
- 12 rows more that the length of unique overlap stable_ID list

### from GRCh38 v97 file with 38,719 rows
- 22,203 match stableID overlap key
- 16,516 did not match
- 10 rows more than length of unique overlapping stable IDs

```python

# drop all rows that have FALSE for match 
TF37.drop(TF37[TF37["37match38stable_id"] == "False"].index, inplace = True)
TF38.drop(TF38[TF38["37match38stable_id"] == "False"].index, inplace = True)
# saving point 
TF37.to_csv("TF37_22205.csv",index=False)
TF38.to_csv("TF38_22203.csv",index=False)

```

## Creating Version Difference TABLE data!!!
#### how many of the 37 and 38 stable matched IDs have different version of ENSG, ENST, and ENSP??

```python
# load saved files 
TF37 = pd.read_csv("TF37_22205.csv")
TF38 = pd.read_csv("TF38_22203.csv")

describeMe(TF37)
describeMe(TF38)

###### results describeME ######
(22205, 11)
Index(['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Protein stable ID',
       'Protein stable ID version', 'CCDS ID', 'UniProtKB/Swiss-Prot ID',
       'Transcript having exact match between ENSEMBL and HAVANA ID',
       'stable_id', '37match38stable_id'],
      dtype='object')
Gene stable ID                                                 0
Gene stable ID version                                         0
Transcript stable ID                                           0
Transcript stable ID version                                   0
Protein stable ID                                              0
Protein stable ID version                                      0
CCDS ID                                                        0
UniProtKB/Swiss-Prot ID                                        0
Transcript having exact match between ENSEMBL and HAVANA ID    0
stable_id                                                      0
37match38stable_id                                             0
dtype: int64
(22203, 10)
Index(['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Protein stable ID',
       'Protein stable ID version', 'CCDS ID', 'UniProtKB/Swiss-Prot ID',
       'stable_id', '37match38stable_id'],
      dtype='object')
Gene stable ID                  0
Gene stable ID version          0
Transcript stable ID            0
Transcript stable ID version    0
Protein stable ID               0
Protein stable ID version       0
CCDS ID                         0
UniProtKB/Swiss-Prot ID         0
stable_id                       0
37match38stable_id              0
dtype: int64
##########

# what rows have duplicate stable ID
dup37stable = TF37[TF37.duplicated(['stable_id'], keep=False)] 
dup37stable = dup37stable.sort_values(['stable_id'],ascending=True)



```
## problem stems from using only ENSG_ENST_ENSP keyID...

## TF37: 
### 6 unique stable IDs point to 18 unique UKB IDs

#### redundancy is in pointing to UKB

```python 
# what rows have duplicate stable ID
dup38stable = TF38[TF38.duplicated(['stable_id'], keep=False)] 
dup38stable = dup38stable.sort_values(['stable_id'],ascending=True)


```
## TF38:
### 4 unique stable IDs point to 14 unique UKB IDs

#### redundancy is in pointing to UKB

```python

# saving duplicated rows
dup38stable.to_csv("v97StableIDmathchedGRCh38_duplicateStableID_fromUniqueUKBcrossRef_14.csv",index=False)
dup37stable.to_csv("v97StableIDmathchedGRCh37_duplicateStableID_fromUniqueUKBcrossRef_18.csv",index=False)

# dropping none stable or version ID rows to reduce redundancy
ID = ['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Protein stable ID',
       'Protein stable ID version','stable_id']

# simplify df to contain less columns, only have stable/versioned from ensembl
ID37only = TF37[ID].copy()
ID38only = TF38[ID].copy()
# check shape 
print(ID37only.shape)
print(ID38only.shape)

# dropping all duplicates keeping only unique rows, and 1st occurence of duplicate
ID37only.drop_duplicates(inplace=True)
ID38only.drop_duplicates(inplace=True)
print(ID37only.shape)
print(ID38only.shape)
# (22193, 7)
# (22193, 7)

# merge on stable id to compare version differences
mergeSTABLE = pd.merge(ID37only,ID38only,how='inner',on=['stable_id'])
```
## merge info: 
#### x = ID37
#### y = ID38

```python

keepme = ['Gene stable ID version_x',
        'Transcript stable ID version_x',
       'Protein stable ID version_x', 'stable_id',
       'Gene stable ID version_y',
        'Transcript stable ID version_y',
     'Protein stable ID version_y']

versionDiff = mergeSTABLE[keepme].copy()
versionDiff
```
####results versionDiff #######

**Gene stable ID version_x	Transcript stable ID version_x	Protein stable ID version_x	stable_id	Gene stable ID version_y	Transcript stable ID version_y	Protein stable ID version_y**

0	ENSG00000230031.6	ENST00000454856.4	ENSP00000456953.1	ENSG00000230031_ENST00000454856_ENSP00000456953	ENSG00000230031.10	ENST00000454856.4	ENSP00000456953.1

##############

## adding columns to quantify number of version ID differences when stable ID key is the same between 37 and 38 for v97 release


```python
keepme = ['Gene stable ID version_x',
        'Transcript stable ID version_x',
       'Protein stable ID version_x', 'stable_id',
       'Gene stable ID version_y',
        'Transcript stable ID version_y',
     'Protein stable ID version_y']


# different ENSG
versionDiff['ENSGv_diff'] = np.where(versionDiff['Gene stable ID version_x'] == versionDiff['Gene stable ID version_y'], 'False', 'True')

checkColumnValues(versionDiff,"ENSGv_diff")
### result checkColumnValues ####
  ENSGv_diff  Count
0       True  21213
1      False    980


# different ENST
versionDiff['ENSTv_diff'] = np.where(versionDiff['Transcript stable ID version_x'] == versionDiff['Transcript stable ID version_y'], 
                                           'False', 'True')
checkColumnValues(versionDiff,"ENSTv_diff")
### result checkColumnValues ####
  ENSTv_diff  Count
0       True  19344
1      False   2849


# different ENSP

# different ENST

versionDiff['ENSPv_diff'] = np.where(versionDiff['Protein stable ID version_x'] == versionDiff['Protein stable ID version_y'], 
                                           'False', 'True')
checkColumnValues(versionDiff,"ENSPv_diff")
### result checkColumnValues ####
ENSPv_diff  Count
0      False  22183
1       True     10


# which proteins have changed
ENSPdiff = versionDiff[versionDiff['ENSPv_diff'] == 'True']

ENSPdiff.to_csv("v97sameStableIDGRCh37and38_diffENSPv_10.csv",index=False)
versionDiff.to_csv("v97versionDifferences_withSameStableID_GRCh37and38_22193.csv",index=False)
```

## GRABBING which stable ENSP correspond to UKB entries

```python

# import entry of ukb IDs in CCDS set
ukbccds = pd.read_csv("entryUniProtCCDSset_simple.txt",sep='\t')
labeledEver = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/all3991_everLabeled_entries_simple.csv",sep='\t')


ukbID = list(ukbccds.Entry)

labeled = list(labeledEver.Entry)

print(len(ukbID)) 

print(len(labeled)) # 3991


# label in entry in my ukbCCDS set
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    return df


myUKBTF38 = addcolumnconditional(ukbID, TF38,'UniProtKB/Swiss-Prot ID','inUKBCCDSset')

labeledTF38 = addcolumnconditional(labeled, TF38,'UniProtKB/Swiss-Prot ID','inLabeledNatureData')

checkColumnValues(myUKBTF38,'inUKBCCDSset')
#### results checkColumnValues #####
 inUKBCCDSset  Count
0         True  22120
1        False     83

checkColumnValues(labeledTF38,'inLabeledNatureData')
#### results checkColumnValues #####
 inLabeledNatureData  Count
0               False  16842
1                True   5361

inUKB = myUKBTF38[myUKBTF38['inUKBCCDSset'] == 'True']
print(inUKB.shape)


inLabel = labeledTF38[labeledTF38['inLabeledNatureData'] == 'True']
print(inLabel.shape)

keepme = ['Protein stable ID','Protein stable ID version', 'CCDS ID', 'UniProtKB/Swiss-Prot ID',
       'stable_id', 'inUKBCCDSset']


simpleinUKB = inUKB[keepme].copy()
simpleinLabeled = inLabel[keepme].copy()


# how many unique entries in this dataframe, if there are 18432 then i have all of the ccds set accounted for
len(simpleinUKB['UniProtKB/Swiss-Prot ID'].unique())
# 15745

len(simpleinLabeled['UniProtKB/Swiss-Prot ID'].unique())
# 3636

---------------
# checking that i know how unique works
# Create an example dataframe
data = {'name': ['Jason', 'Jason', 'Tina', 'Jake', 'Amy'], 
        'year': [2012, 2012, 2013, 2014, 2014], 
        'reports': [4, 24, 31, 2, 3]}
df = pd.DataFrame(data, index = ['Cochice', 'Pima', 'Santa Cruz', 'Maricopa', 'Yuma'])
df.name.unique()

array(['Jason', 'Tina', 'Jake', 'Amy'], dtype=object
 --------------

```
## not all of my UKBccds set entries are in the TF38 file that only contained entries matched to 37 on ENSG_ENST_ENSP 


## not all the entries we have labeled in Nature paper are in TF38 stable ID key matched dataset, out of 3991 only 3636 are there

---

## creating entry to multi ENSP ID mapping dictionaries: 

```python
# i need to count unique ENSP 'ID' stable in every UniProt entry 'domain' 
# entryID  Count
# Q4453 .   4 . means 4 unique stable IDs map to that entry

entry2ENSPcount = simpleinUKB.groupby('UniProtKB/Swiss-Prot ID')['Protein stable ID'].nunique()
######result######
entry2ENSPcount
A0A183    1
A0AUZ9    1
A0AV02    2
A0AV96    2
A0AVF1    2
A0AVI4    2
A0AVK6    1
A0AVT1    1
A0FGR9    1
A0JNW5    2
A0PG75    1
Name: Protein stable ID, Length: 15745, dtype: int64

# i need to count unique ENSP 'ID' stable in every UniProt entry ever labeled'domain' 
# entryID  Count
# Q4453 .   4 . means 4 unique stable IDs map to that entry

labeled2ENSPcount = simpleinLabeled.groupby('UniProtKB/Swiss-Prot ID')['Protein stable ID'].nunique()

labeled2ENSPcount
######result######
UniProtKB/Swiss-Prot ID
A0AVT1    1
A0JNW5    2
A1KXE4    1
A1L0T0    1
A2RUC4    1
A4D1E9    2
A5YKK6    2
A6NC98    1
A6NDG6    1
Name: Protein stable ID, Length: 3636, dtype: int64

# saving files
entry2ENSPcount.to_csv("UKBccdsEntry2stableENSP_fromTF38_stableIDkeyMatched_15745entries_counts2ENSPmultimapping.csv")

labeled2ENSPcount.to_csv("allLabeledBeforeEntries2stableENSP_fromTF38_stableIDkeyMatched_3636entries_counts2ENSPmultimapping.csv")


```



---
---
---
---
---
---
---
---

# cells of second [correct key id 2.0] attempt below: 

In [4]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv

In [5]:
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

In [6]:
sys.path.append('/Users/mariapalafox/Desktop/Toolbox')
from all_funx import *

In [7]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/BIOMART_ENSEMBL/")
print(os.listdir())

['TF_38_truelabeled_before_5361_keyid2.csv', '.DS_Store', 'BIOMART_notes_1stattempt_keyID.txt', 'TF_38_22202_keyid_v2.csv', 'NOTES.txt', 'GRCh37vsGRCh38_ensembl.sh', 'biomartGRCh38p.12_v97_filterCCDSIDonly.txt', 'entryUniProtCCDSset_simple.txt', 'StableIDs_allMatched_btw_GRCh37_38_ensemblv97_22202.csv', 'mapping_37.txt', 'biomart_CCDSIDonly_GRCh37_v97_peptideFasta.txt', 'Homo_sapiens.GRCh38.pep.all.fa', 'GRCh37vsGRCh38_ensembl copy.sh', 'mapping_38.txt', 'TF_37_truelabeled_before_5361_keyid2.csv', 'biomartGRCh37p.13_v97_filterCCDSIDonly.txt', 'TF_37_22202_keyid_v2.csv', 'biomartGRCh37p.13_v97_filterHAVANAensemblTranscriptMatchOnly.txt']


---
FROM 19_09_04_biomart_ensembl_GRCh37vs38...

##### Review: CREATED new keyID 'ENSG_ENST_ENSP_UKBSP_CCDS'

```bash
# review
X = 37
Y = 38
inner merge on ENST stable, compared ENSG/ENSP/UKBswissProt/CCDS all stable for v97 ensembl downloaded from biomart
```

```python
checkMatch = pd.read_csv("StableIDs_allMatched_btw_GRCh37_38_ensemblv97_22202.csv")
# create stable_id for mapping to stableMatch IDs to versioned rows in 37 and 38 file 
# extension of original key
# does not matter than some have _x and some have _y, they are identical
checkMatch['stable_id2'] = checkMatch['Gene stable ID_x'].astype(str) + \
'_' + checkMatch['ENST'].astype(str) + \
'_' + checkMatch['Protein stable ID_y'].astype(str) + \
'_' + checkMatch['CCDS ID_y'] + \
'_' + checkMatch['UniProtKB/Swiss-Prot ID_y']
# example: 'ENSG00000168675_ENST00000399848_ENSP00000382741_CCDS32794_O15165'
# added this ID to version 37 and version 38 files
# get total # of unique UKB IDs
print(len(checkMatch.stable_id2.unique()))
print(len(version37.stable_id2.unique()))
print(len(version38.stable_id2.unique()))
# all the overlapping IDs between GRCh37 and GRCh38 are unique
22202
23334
38719

38719-22202 = 16,517
# diff between v97 38 and overlap len of 37 w/ 38
23334-22202 = 1132
# diff btw v97 37 with keyid2
checkMatchlist = checkMatch['stable_id2'].tolist()
print(len(checkMatchlist))
22202
# label each 37 and 38 version file with True False 
# if stable_id is in the unique set of ID from my stable ID overlap between assemblies analysis
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    return df
TF_37 = addcolumnconditional(checkMatchlist, version37, "stable_id2", "37match38stable_ids_2")
37match38stable_ids_2  Count
0                  True  22202
1                 False   1132
TF_38 = addcolumnconditional(checkMatchlist, version38, "stable_id2", "37match38stable_ids_2")
37match38stable_ids_2  Count
0                  True  22202
1                 False  16517
# total 22,202 unique stable_id2 overlapping 37 and 38 
### from GRCh37 v97 file with 23,334 rows & GRCh38 v97 file with 38,719 rows
> 22,202 match stableID overlap key
NUMBER of unique Uniprot swissprot stable IDs
> 15819

NUMBER of unique CCDS stable IDs
> 21070

NUMBER of unique protein stable IDs
> 22193

# saving all overlapping 37 and 38 IDs
TF_37.to_csv("TF_37_22202_keyid_v2.csv",index=False)
TF_38.to_csv("TF_38_22202_keyid_v2.csv",index=False)
# filtere for entries in labeled ever dataset
 Labeled_before_entry  Count
0                False  16841
1                 True   5361
  Labeled_before_entry  Count
0                False  16841
1                 True   5361
# starting point files: 
true37label2.to_csv("TF_37_truelabeled_before_5361_keyid2.csv", index=False)
true38label.to_csv("TF_38_truelabeled_before_5361_keyid2.csv",index=False)

```

# TODO-0: CREATE version ID difference table from 5,361 rows sharing ENSG_ENST_ENSP_CCDS_UKB between v97 GRCh37 and GRCh38 biomart files


In [8]:
diff37 = pd.read_csv("TF_37_truelabeled_before_5361_keyid2.csv")
diff38 = pd.read_csv("TF_38_truelabeled_before_5361_keyid2.csv")

In [9]:
diff37.columns

Index(['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Protein stable ID',
       'Protein stable ID version', 'CCDS ID', 'UniProtKB/Swiss-Prot ID',
       'stable_id2'],
      dtype='object')

In [10]:
diff37.head(3)

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Protein stable ID,Protein stable ID version,CCDS ID,UniProtKB/Swiss-Prot ID,stable_id2
0,ENSG00000001497,ENSG00000001497.12,ENST00000374804,ENST00000374804.5,ENSP00000363937,ENSP00000363937.5,CCDS55433,Q9Y4W2,ENSG00000001497_ENST00000374804_ENSP0000036393...
1,ENSG00000001497,ENSG00000001497.12,ENST00000374807,ENST00000374807.5,ENSP00000363940,ENSP00000363940.5,CCDS55434,Q9Y4W2,ENSG00000001497_ENST00000374807_ENSP0000036394...
2,ENSG00000001497,ENSG00000001497.12,ENST00000374811,ENST00000374811.3,ENSP00000363944,ENSP00000363944.3,CCDS14381,Q9Y4W2,ENSG00000001497_ENST00000374811_ENSP0000036394...


In [11]:
diff38.head(3)

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Protein stable ID,Protein stable ID version,CCDS ID,UniProtKB/Swiss-Prot ID,stable_id2
0,ENSG00000001497,ENSG00000001497.16,ENST00000374804,ENST00000374804.9,ENSP00000363937,ENSP00000363937.5,CCDS55433,Q9Y4W2,ENSG00000001497_ENST00000374804_ENSP0000036393...
1,ENSG00000001497,ENSG00000001497.16,ENST00000374807,ENST00000374807.9,ENSP00000363940,ENSP00000363940.5,CCDS55434,Q9Y4W2,ENSG00000001497_ENST00000374807_ENSP0000036394...
2,ENSG00000001497,ENSG00000001497.16,ENST00000374811,ENST00000374811.7,ENSP00000363944,ENSP00000363944.3,CCDS14381,Q9Y4W2,ENSG00000001497_ENST00000374811_ENSP0000036394...


In [12]:
diff37.nunique()

Gene stable ID                  3642
Gene stable ID version          3642
Transcript stable ID            5361
Transcript stable ID version    5361
Protein stable ID               5361
Protein stable ID version       5361
CCDS ID                         5049
UniProtKB/Swiss-Prot ID         3636
stable_id2                      5361
dtype: int64

In [13]:
diff38.nunique()

Gene stable ID                  3642
Gene stable ID version          3642
Transcript stable ID            5361
Transcript stable ID version    5361
Protein stable ID               5361
Protein stable ID version       5361
CCDS ID                         5049
UniProtKB/Swiss-Prot ID         3636
stable_id2                      5361
dtype: int64

In [14]:
# merge on stable_id2
keepme = ['Gene stable ID version_x',
        'Transcript stable ID version_x',
       'Protein stable ID version_x', 'stable_id2',
       'Gene stable ID version_y',
        'Transcript stable ID version_y',
     'Protein stable ID version_y']

versionDiff  = pd.merge(diff37,diff38,how='inner',on=['stable_id2'])
print(versionDiff.columns)
print(versionDiff.shape)

Index(['Gene stable ID_x', 'Gene stable ID version_x',
       'Transcript stable ID_x', 'Transcript stable ID version_x',
       'Protein stable ID_x', 'Protein stable ID version_x', 'CCDS ID_x',
       'UniProtKB/Swiss-Prot ID_x', 'stable_id2', 'Gene stable ID_y',
       'Gene stable ID version_y', 'Transcript stable ID_y',
       'Transcript stable ID version_y', 'Protein stable ID_y',
       'Protein stable ID version_y', 'CCDS ID_y',
       'UniProtKB/Swiss-Prot ID_y'],
      dtype='object')
(5361, 17)


### x = GRCh37 v97 file
### y = GRCh38 v97 file

In [18]:
# different ENSG
gdiff = versionDiff[['Gene stable ID version_x', 'Gene stable ID version_y']].copy()
gdiff.drop_duplicates(keep='first',inplace=True) 

gdiff['ENSGv_diff'] = np.where(gdiff['Gene stable ID version_x'] != gdiff['Gene stable ID version_y'], 'True', 'False')
checkColumnValues(gdiff,"ENSGv_diff")

  ENSGv_diff  Count
0       True   3607
1      False     35


In [19]:
# different ENST
tdiff = versionDiff[['Transcript stable ID version_x', 'Transcript stable ID version_y']].copy()
tdiff.drop_duplicates(keep='first',inplace=True) 
tdiff['ENSTv_diff'] = np.where(tdiff['Transcript stable ID version_x'] != tdiff['Transcript stable ID version_y'], 
                                           'True', 'False')
checkColumnValues(tdiff,"ENSTv_diff")

  ENSTv_diff  Count
0       True   5017
1      False    344


In [20]:
# different ENSP
pdiff = versionDiff[['Protein stable ID version_x', 'Protein stable ID version_y']].copy()
pdiff.drop_duplicates(keep='first',inplace=True) 

pdiff['ENSPv_diff'] = np.where(pdiff['Protein stable ID version_x'] != pdiff['Protein stable ID version_y'], 'True', 'False')
checkColumnValues(pdiff,"ENSPv_diff")

  ENSPv_diff  Count
0      False   5361


---
---
# ALL TODO's outlined below:
---

# TODO-1: compare mapping between UniProt and Ensembl Stable IDs from 2 different SOURCE [UniProt ID mapping || biomart v97 overlapped btw assemblies mapping] 



### GOAL: 
- show how UniProt Isoform <---> Ensembl stable IDs either solves or does not solve the multiple mapping issue seen with mapping in SOURCE biomart v97 files Ensembl stable IDs <---> UniProt stable IDs 
- will also provide support for canonical sequence not having isoform specific entry!

### input files: 

### output files: 

# TODO-2: create dictionary of UKB ID to multiple ENSP will show if isoform details solves multi mapping of ENSP to same UKB ID problem

```python
# first attemp gene dictionary files were named: 
mapping from UKB entry to ENSP dictionaries in files below: 
- "allLabeledBeforeEntries2stableENSP_fromTF38_stableIDkeyMatched_3636entries_counts2ENSPmultimapping.csv”
- "UKBccdsEntry2stableENSP_fromTF38_stableIDkeyMatched_15745entries_counts2ENSPmultimapping.csv"
```

---
---
---

from README_ID_MAPPING.txt...

```bash

Universal Protein Resource (UniProt)
====================================
https://www.uniprot.org/downloads
7/31/19  7:00:00 AM was last update from the file that i downloaded
downloaded 8/5/19

The Universal Protein Resource (UniProt), a collaboration between the European
Bioinformatics Institute (EBI), the SIB Swiss Institute of Bioinformatics, and
the Protein Information Resource (PIR), is comprised of three databases, each
optimized for different uses. The UniProt Knowledgebase (UniProtKB) is the
central access point for extensively curated protein information, including
function, classification and cross-references. The UniProt Reference Clusters
(UniRef) combine closely related sequences into a single record to speed up
sequence similarity searches. The UniProt Archive (UniParc) is a comprehensive
repository of all protein sequences, consisting only of unique identifiers and
sequences.



IDMAPPING
=========

This directory, databases/uniprot/current_release/knowledgebase/idmapping/,
contains the idmapping data files which are updated in conjunction with the UniProt 
Knowledgebase (UniProtKB). Whenever available the mappings are extracted from the 
UniProtKB records. 

All files listed below contain the complete data sets corresponding to the
most recent release.

1) idmapping.dat
This file has three columns, delimited by tab:
1. UniProtKB-AC 
2. ID_type 
3. ID
where ID_type is the database name as appearing in UniProtKB cross-references, 
and as supported by the ID mapping tool on the UniProt web site, 
http://www.uniprot.org/mapping and where ID is the identifier in 
that cross-referenced database.
    
2) idmapping_selected.tab
We also provide this tab-delimited table which includes
the following mappings delimited by tab:

1. UniProtKB-AC
2. UniProtKB-ID
3. GeneID (EntrezGene)
4. RefSeq
5. GI
6. PDB
7. GO
8. UniRef100
9. UniRef90
10. UniRef50
11. UniParc
12. PIR
13. NCBI-taxon
14. MIM
15. UniGene
16. PubMed
17. EMBL
18. EMBL-CDS
19. Ensembl
20. Ensembl_TRS
21. Ensembl_PRO
22. Additional PubMed
3) example files

idmapping_selected.tab.example has the first 1000 lines from idmapping_selected.tab
idmapping.dat.example has the first 10,000 lines from idmapping.dat

4) We provide separate ID mapping tables for selected model organisms in subdirectory: by_organism

5) idmapping.dat.2015_03 and idmapping_selected.tab.2015_03
These are archived versions of the files idmapping.dat and idmapping_selected.tab, respectively, for release 2015_03.
This was the last release before proteome redundancy reduction (http://www.uniprot.org/help/proteome_redundancy) which 
caused the size of UniProtKB/TrEMBL to drop from 92 million to 47 million entries. Users trying to map obsolete identifiers
to external databases and vice versa may find these files useful.

6) README
This file.

The /complete/docs subdirectory contains various UniProt documents.

--------------------------------------------------------------------------------
  LICENSE
--------------------------------------------------------------------------------
We have chosen to apply the Creative Commons Attribution (CC BY 4.0) License
(https://creativecommons.org/licenses/by/4.0/) to all copyrightable parts of
our databases.

(c) 2002-2019 UniProt Consortium

--------------------------------------------------------------------------------
  DISCLAIMER
--------------------------------------------------------------------------------
We make no warranties regarding the correctness of the data, and disclaim
liability for damages resulting from its use. We cannot provide unrestricted
permission regarding the use of the data, as some data may be covered by patents
or other rights.

Any medical or genetic information is provided for research, educational and
informational purposes only. It is not in any way intended to be used as a
substitute for professional medical advice, diagnosis, treatment or care.

```

# TODO-1: compare mapping between UniProt and Ensembl Stable IDs from 2 different SOURCE [UniProt ID mapping || biomart v97 overlapped btw assemblies mapping] 

#### GOAL: 
- show how UniProt Isoform <---> Ensembl stable IDs either solves or does not solve the multiple mapping issue seen with mapping in SOURCE biomart v97 files Ensembl stable IDs <---> UniProt stable IDs 
- will also provide support for canonical sequence not having isoform specific entry!

### input files: 
##### SOURCE UniProt mapping services
- [pre-grep file] HUMAN_9606_idmapping.dat
- HUMAN_9606_idmapping_grepEnsembl.txt

##### SOURCE [projectID: v97 GRCh37 & GRCh38 stable key ID matched version differences/stability test of Ensembl DNA RNA PRO]
- TF_38_truelabeled_before_5361_keyid2.csv
- TF_37_22202_keyid_v2.csv
- TF_37_truelabeled_before_5361_keyid2.csv
- TF_38_22202_keyid_v2.csv



##### output files: 

In [1]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

sys.path.append('/Users/mariapalafox/Desktop/Toolbox')
from all_funx import *

os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/DEPENDENTvsINDEPENDENT/")
print(os.listdir())


['TF_38_truelabeled_before_5361_keyid2.csv', 'TF_38_22202_keyid_v2.csv', 'HUMAN_9606_idmapping_grepEnsembl.txt', 'README_ID_mapping.txt', 'TF_37_truelabeled_before_5361_keyid2.csv', 'TF_37_22202_keyid_v2.csv']


In [25]:
ukb_idmap = "HUMAN_9606_idmapping_grepEnsembl.txt"
#path = "/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/HUMAN_9606_uniprot_IDmapping/"

uni = pd.read_csv(ukb_id_map, sep='\t', header=None)

In [26]:
uni.head(3)

,0,1,2
0,P31946,Ensembl,ENSG00000166913
1,P31946-1,Ensembl_TRS,ENST00000353703
2,P31946-1,Ensembl_PRO,ENSP00000300161


In [27]:
describeMe(uni)

(294943, 3)
Int64Index([0, 1, 2], dtype='int64')
0    0
1    0
2    0
dtype: int64


In [28]:
uni.columns = ['entry','Source_type','stableID']
uni.head(3)

,entry,Source_type,stableID
0,P31946,Ensembl,ENSG00000166913
1,P31946-1,Ensembl_TRS,ENST00000353703
2,P31946-1,Ensembl_PRO,ENSP00000300161


In [29]:
transx = uni.head(3)
newf = transx.pivot(index='entry', columns='Source_type')
newf

stableID                                  
Source_type          Ensembl      Ensembl_PRO      Ensembl_TRS
entry                                                         
P31946       ENSG00000166913              NaN              NaN
P31946-1                 NaN  ENSP00000300161  ENST00000353703

In [30]:
# oldcols = {'col1':['a','a','b','b'], 'col2':['c','d','c','d'], 'col3':[1,2,3,4]}
# a = pd.DataFrame(oldcols)  
# print(a)
newf = a.pivot(index='col1', columns='col2')
print()
print(newf)


     col3   
col2    c  d
col1        
a       1  2
b       3  4


#### for pivot later to drop level name
newf.columns = newf.columns.droplevel(0)

### pivot below failed because entry is not unique, therefore can not be index

In [32]:
pivUKB = uni.pivot(index='entry', columns='Source_type', values='stableID')
pivUKB


ValueError: Index contains duplicate entries, cannot reshape

https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

https://stackoverflow.com/questions/22173572/change-column-values-to-column-headers-in-pandas

[best help] https://hackernoon.com/reshaping-data-in-python-fa27dda2ff77

In [31]:
uni

,entry,Source_type,stableID
0,P31946,Ensembl,ENSG00000166913
1,P31946-1,Ensembl_TRS,ENST00000353703
2,P31946-1,Ensembl_PRO,ENSP00000300161
3,P31946-1,Ensembl_TRS,ENST00000372839
4,P31946-1,Ensembl_PRO,ENSP00000361930
5,P62258,Ensembl,ENSG00000108953
6,P62258-1,Ensembl_TRS,ENST00000264335
7,P62258-1,Ensembl_PRO,ENSP00000264335
8,P62258-2,Ensembl_TRS,ENST00000571732
9,P62258-2,Ensembl_PRO,ENSP00000461762


In [40]:
# stacking and unstacking like concept of pivot but for multi index
unimulti = uni.set_index(['entry', 'Source_type'])
unimulti.head(10)

stableID
entry    Source_type                 
P31946   Ensembl      ENSG00000166913
P31946-1 Ensembl_TRS  ENST00000353703
         Ensembl_PRO  ENSP00000300161
         Ensembl_TRS  ENST00000372839
         Ensembl_PRO  ENSP00000361930
P62258   Ensembl      ENSG00000108953
P62258-1 Ensembl_TRS  ENST00000264335
         Ensembl_PRO  ENSP00000264335
P62258-2 Ensembl_TRS  ENST00000571732
         Ensembl_PRO  ENSP00000461762

In [42]:
unigroup = uni.groupby(['entry','Source_type'])['stableID'].apply(';'.join).reset_index()
# groupby sets the index hierarchy to 1st entry, then source_type...the values are noted by [] with funxs to merge w
# sep as ; finally reset index!

In [43]:
unigroup

,entry,Source_type,stableID
0,A0A024QYW1,Ensembl,ENSG00000206140
1,A0A024QYW1,Ensembl_PRO,ENSP00000490356
2,A0A024QYW1,Ensembl_TRS,ENST00000545656
3,A0A024QZ18,Ensembl,ENSG00000269313
4,A0A024QZ18,Ensembl_PRO,ENSP00000481198
5,A0A024QZ18,Ensembl_TRS,ENST00000616266
6,A0A024QZ33,Ensembl,ENSG00000126653
7,A0A024QZ33,Ensembl_PRO,ENSP00000477862
8,A0A024QZ33,Ensembl_TRS,ENST00000612959
9,A0A024QZ42,Ensembl,ENSG00000249915


In [73]:
describeMe(unigroup)

(249069, 4)
Index(['entry', 'Source_type', 'stableID', 'Labeled_before_entry'], dtype='object')
entry                   0
Source_type             0
stableID                0
Labeled_before_entry    0
dtype: int64


In [44]:
294943 - 249069 
# wc -l of file then len after groupby

45874

# 45,874 rows difference = redundancy in UKB Human ID mapping file

In [53]:
# importing entries from labeled ever file containing 3991 entries
labeledEver = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/all3991_everLabeled_entries_simple.csv",sep='\t')
labeled = list(labeledEver.Entry)
print(len(labeled))



3991


In [56]:
# had to change to accomodate the isoform versions in ID mapping

def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if '-' in g:
            # if it has -#, only compare chars before -
            check = g.split('-')[0]
            if check in mapList:
                mendel.append("True")
            else:
                mendel.append("False")
        else:   
            if g in mapList: 
                mendel.append("True")
            else:
                mendel.append("False")
                
    df.loc[:,newcol] = mendel
    
    return df

In [57]:
# running laberler funx to filter for only rows that have UKBID also included in Nature experiment unique entries list
unilabeled = addcolumnconditional(labeled, unigroup,'entry','Labeled_before_entry')
checkColumnValues(unilabeled, 'Labeled_before_entry')

  Labeled_before_entry   Count
0                False  229912
1                 True   19157


In [58]:
trueunilabeled= unilabeled[unilabeled['Labeled_before_entry'] == 'True']

print(trueunilabeled.shape)

(19157, 4)


In [61]:
trueunilabeled.head(10)

# (19157, 4)


,entry,Source_type,stableID,Labeled_before_entry
36466,A0AVT1,Ensembl,ENSG00000033178,True
36467,A0AVT1-1,Ensembl_PRO,ENSP00000313454,True
36468,A0AVT1-1,Ensembl_TRS,ENST00000322244,True
36469,A0AVT1-3,Ensembl_PRO,ENSP00000399234,True
36470,A0AVT1-3,Ensembl_TRS,ENST00000420827,True
36471,A0FGR8,Ensembl,ENSG00000117868,True
36472,A0FGR8-2,Ensembl_PRO,ENSP00000499020,True
36473,A0FGR8-2,Ensembl_TRS,ENST00000652148,True
36494,A0JNW5,Ensembl,ENSG00000111647,True
36495,A0JNW5-1,Ensembl_PRO,ENSP00000279907,True


In [62]:
ensp_labeled = trueunilabeled[trueunilabeled['Source_type'] == "Ensembl_PRO"]
describeMe(ensp_labeled)

(7601, 4)
Index(['entry', 'Source_type', 'stableID', 'Labeled_before_entry'], dtype='object')
entry                   0
Source_type             0
stableID                0
Labeled_before_entry    0
dtype: int64


In [75]:
ensp_labeled.reset_index(drop=True,inplace=True)
ensp_labeled

,entry,Source_type,stableID,Labeled_before_entry,entry_stable
0,A0AVT1-1,Ensembl_PRO,ENSP00000313454,True,A0AVT1
1,A0AVT1-3,Ensembl_PRO,ENSP00000399234,True,A0AVT1
2,A0FGR8-2,Ensembl_PRO,ENSP00000499020,True,A0FGR8
3,A0JNW5-1,Ensembl_PRO,ENSP00000279907,True,A0JNW5
4,A0JNW5-2,Ensembl_PRO,ENSP00000349285,True,A0JNW5
5,A0MZ66-1,Ensembl_PRO,ENSP00000347532,True,A0MZ66
6,A0MZ66-2,Ensembl_PRO,ENSP00000480109,True,A0MZ66
7,A0MZ66-4,Ensembl_PRO,ENSP00000376636,True,A0MZ66
8,A0MZ66-5,Ensembl_PRO,ENSP00000260777,True,A0MZ66
9,A0MZ66-8,Ensembl_PRO,ENSP00000376635,True,A0MZ66


In [76]:
len(set(ensp_labeled['entry'].tolist()))

7601

In [67]:
# making stableentry column 
ensp_labeled['entry_stable'] = ensp_labeled.entry.apply(lambda row: row.split("-")[0])



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
ensp_labeled.head(15)

,entry,Source_type,stableID,Labeled_before_entry,entry_stable
0,A0AVT1-1,Ensembl_PRO,ENSP00000313454,True,A0AVT1
1,A0AVT1-3,Ensembl_PRO,ENSP00000399234,True,A0AVT1
2,A0FGR8-2,Ensembl_PRO,ENSP00000499020,True,A0FGR8
3,A0JNW5-1,Ensembl_PRO,ENSP00000279907,True,A0JNW5
4,A0JNW5-2,Ensembl_PRO,ENSP00000349285,True,A0JNW5
5,A0MZ66-1,Ensembl_PRO,ENSP00000347532,True,A0MZ66
6,A0MZ66-2,Ensembl_PRO,ENSP00000480109,True,A0MZ66
7,A0MZ66-4,Ensembl_PRO,ENSP00000376636,True,A0MZ66
8,A0MZ66-5,Ensembl_PRO,ENSP00000260777,True,A0MZ66
9,A0MZ66-8,Ensembl_PRO,ENSP00000376635,True,A0MZ66


In [70]:
# how many stable IDs in df
entrylist_stable = ensp_labeled['entry_stable'].tolist()
entryset = set(entrylist_stable)
len(entryset)

3955

In [71]:
entryset

{'P60900',
 'O95793',
 'P07108',
 'P13674',
 'Q14669',
 'Q15758',
 'Q92734',
 'Q9GZZ9',
 'Q16555',
 'Q96T76',
 'O43681',
 'Q8IUY3',
 'P50395',
 'Q06587',
 'Q96HR3',
 'Q9BZE4',
 'P84022',
 'P61353',
 'Q13151',
 'P08238',
 'Q9Y3C1',
 'P61006',
 'O75940',
 'Q9UL46',
 'Q92600',
 'Q9Y5T5',
 'Q6UN15',
 'Q9NR56',
 'Q92820',
 'Q8TDX7',
 'P78344',
 'Q99538',
 'O60216',
 'P47755',
 'P17676',
 'Q14974',
 'O15145',
 'Q9NZN8',
 'Q8N3U4',
 'Q8WXG6',
 'P46940',
 'P41091',
 'P83731',
 'Q86UX7',
 'Q8WVT3',
 'P14735',
 'Q9UPN3',
 'Q9UN37',
 'Q96T51',
 'Q96JY6',
 'Q86WB0',
 'Q9P2X3',
 'P67936',
 'Q9UPQ0',
 'O75817',
 'Q8N3X1',
 'P13984',
 'Q96IZ0',
 'P28482',
 'Q9Y5L0',
 'Q9Y5V0',
 'P07686',
 'Q99638',
 'Q6WKZ4',
 'Q9NPJ6',
 'P09543',
 'Q92575',
 'Q6UXN9',
 'Q8NDI1',
 'P50748',
 'O43172',
 'Q5VTL8',
 'Q8TAQ9',
 'Q712K3',
 'P24666',
 'P46926',
 'Q9UKT9',
 'P84098',
 'P61803',
 'P35579',
 'Q8ND04',
 'Q9Y6G9',
 'O60502',
 'Q9UGJ1',
 'Q659C4',
 'O60568',
 'Q01970',
 'Q99590',
 'Q14061',
 'Q8N6N3',
 'Q9BRX5',

# 3955 out of 3991 ever labeled before uniprot entries are in the 
1. UKB human mapping file with 
2. a ENSP source_type and 
3. a formatted isoform entry ID that equals a stable entry from canonical ukb labeled ever data 

In [72]:
# how many of these unique entries have more than 1 isoform linked entry in file
isoCount = ensp_labeled.loc[:,'entry_stable'].value_counts()
isoCount

O60716    21
Q03060    20
Q9UKT9    15
P15812    13
Q13444    12
Q9ULU4    12
P29590    11
O00499    11
Q8TEW0     9
Q15149     9
P04637     9
Q86UA6     8
Q15366     8
Q5T5P2     8
Q8WXG6     8
Q07866     8
Q96PU5     8
Q04637     8
O75334     8
Q9UHD8     8
P56945     8
Q9H4L5     8
Q9NUL3     8
P09493     8
P22694     8
O43768     8
O96017     8
P22307     8
P46937     8
Q9UPQ0     8
          ..
P63173     1
P46063     1
P42704     1
O60759     1
P52292     1
Q96FF9     1
Q6P0N0     1
A6NDU8     1
Q86UY8     1
Q9H773     1
P08567     1
Q9NYU2     1
Q8N8R5     1
Q06830     1
P31146     1
P39687     1
P49336     1
P52732     1
Q9P215     1
Q9BWD1     1
Q9UNH6     1
Q9BUH6     1
Q96EY5     1
Q15437     1
O75879     1
Q9NZ63     1
O43823     1
P98179     1
Q96FZ2     1
Q9NRN9     1
Name: entry_stable, Length: 3955, dtype: int64

In [74]:
print(isoCount.shape)
print(ensp_labeled.shape)
print(trueunilabeled.shape)

(3955,)
(7601, 5)
(19157, 4)


In [77]:
isoCount.to_csv("UniprotIDMAPPING_isoformCount__labeledBefore_ENSPsource_3955unique entries.csv")
ensp_labeled.to_csv("UniprotIDMAPPING_ENSPsourcetypefiltered_labeledBeforeEntry_7601")
trueunilabeled.to_csv("UniprotIDMAPPING_trueunilabeled_incl_ENST_ENSP_19157.csv")

# creating entry to stable IDs from keyID2 37 38 biomart  
## this is not really helpful

d38 = "TF_38_truelabeled_before_5361_keyid2.csv"

d37 = "TF_37_truelabeled_before_5361_keyid2.csv"


In [78]:
d38 = "TF_38_truelabeled_before_5361_keyid2.csv"
d37 = "TF_37_truelabeled_before_5361_keyid2.csv"
d38 = pd.read_csv(d38)
d37 = pd.read_csv(d37)

In [80]:
d37.head(10)

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Protein stable ID,Protein stable ID version,CCDS ID,UniProtKB/Swiss-Prot ID,stable_id2
0,ENSG00000001497,ENSG00000001497.12,ENST00000374804,ENST00000374804.5,ENSP00000363937,ENSP00000363937.5,CCDS55433,Q9Y4W2,ENSG00000001497_ENST00000374804_ENSP0000036393...
1,ENSG00000001497,ENSG00000001497.12,ENST00000374807,ENST00000374807.5,ENSP00000363940,ENSP00000363940.5,CCDS55434,Q9Y4W2,ENSG00000001497_ENST00000374807_ENSP0000036394...
2,ENSG00000001497,ENSG00000001497.12,ENST00000374811,ENST00000374811.3,ENSP00000363944,ENSP00000363944.3,CCDS14381,Q9Y4W2,ENSG00000001497_ENST00000374811_ENSP0000036394...
3,ENSG00000001631,ENSG00000001631.10,ENST00000340022,ENST00000340022.2,ENSP00000344668,ENSP00000344668.2,CCDS5624,O00522,ENSG00000001631_ENST00000340022_ENSP0000034466...
4,ENSG00000001631,ENSG00000001631.10,ENST00000394503,ENST00000394503.2,ENSP00000378011,ENSP00000378011.2,CCDS34679,O00522,ENSG00000001631_ENST00000394503_ENSP0000037801...
5,ENSG00000001631,ENSG00000001631.10,ENST00000412043,ENST00000412043.2,ENSP00000410909,ENSP00000410909.2,CCDS5624,O00522,ENSG00000001631_ENST00000412043_ENSP0000041090...
6,ENSG00000002330,ENSG00000002330.9,ENST00000309032,ENST00000309032.3,ENSP00000309103,ENSP00000309103.3,CCDS8065,Q92934,ENSG00000002330_ENST00000309032_ENSP0000030910...
7,ENSG00000002330,ENSG00000002330.9,ENST00000394532,ENST00000394532.3,ENSP00000378040,ENSP00000378040.3,CCDS8065,Q92934,ENSG00000002330_ENST00000394532_ENSP0000037804...
8,ENSG00000002549,ENSG00000002549.8,ENST00000226299,ENST00000226299.4,ENSP00000226299,ENSP00000226299.4,CCDS3422,P28838,ENSG00000002549_ENST00000226299_ENSP0000022629...
9,ENSG00000002746,ENSG00000002746.10,ENST00000395891,ENST00000395891.2,ENSP00000379228,ENSP00000379228.1,CCDS5469,Q76N89,ENSG00000002746_ENST00000395891_ENSP0000037922...


In [82]:
uniprot2ensp_d37 = d37.groupby(['UniProtKB/Swiss-Prot ID'])['Protein stable ID'].apply(';'.join).reset_index()
# all teh ENSP are uniqeu , 5361

In [83]:
uniprot2ensp_d37

,UniProtKB/Swiss-Prot ID,Protein stable ID
0,A0AVT1,ENSP00000313454
1,A0JNW5,ENSP00000279907;ENSP00000349285
2,A1KXE4,ENSP00000374565
3,A1L0T0,ENSP00000263383
4,A2RUC4,ENSP00000346627
5,A4D1E9,ENSP00000222511;ENSP00000257659
6,A5YKK6,ENSP00000320949;ENSP00000413113
7,A6NC98,ENSP00000349238
8,A6NDG6,ENSP00000330918
9,A6NDU8,ENSP00000371061


# 3,636 unique index entries -- multi map to --- ensembl stable IDS shared between v97 GRCh37 adn GRCh38 


## the v97 groupedby UKBID to ENSP were identical for 37 and 38, QC



In [84]:
# QC compare to 38 which should be identical
uniprot2ensp_d38 = d38.groupby(['UniProtKB/Swiss-Prot ID'])['Protein stable ID'].apply(';'.join).reset_index()
uniprot2ensp_d38.head(3)

,UniProtKB/Swiss-Prot ID,Protein stable ID
0,A0AVT1,ENSP00000313454
1,A0JNW5,ENSP00000279907;ENSP00000349285
2,A1KXE4,ENSP00000374565


In [85]:
uniprot2ensp_d38.equals(uniprot2ensp_d37)

True

In [86]:
uniprot2ensp_d38.to_csv("Uniprot2ENSP_key4versioncheck_CYSLYSmapping_failure_3636unique_entry.csv",index=False)